In [ ]:
# NSGA-II Load Balancing Example
# (Adapte conforme necessário)

from deap import base, creator, tools
import random
import numpy as np

# Parâmetros
NUM_TASKS = 50
NUM_VMS = 5

VM_SPEEDS = [1.0, 1.5, 2.0, 0.8, 1.2]
VM_COSTS = [1.0, 1.8, 3.0, 0.6, 1.3]
TASK_WORKLOADS = [random.randint(5, 20) for _ in range(NUM_TASKS)]

def evaluate_allocation(individual):
    vm_queues = [[] for _ in range(NUM_VMS)]
    for task, vm in enumerate(individual):
        vm_queues[vm].append(task)

    latencies = []
    total_cost = 0
    for vm in range(NUM_VMS):
        t = 0
        total_service = 0
        for task in vm_queues[vm]:
            service = TASK_WORKLOADS[task] / VM_SPEEDS[vm]
            latency = t + service
            latencies.append(latency)
            t += service
            total_service += service
        total_cost += total_service * VM_COSTS[vm]

    return np.mean(latencies), total_cost

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_vm", random.randrange, NUM_VMS)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_vm, n=NUM_TASKS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_allocation)
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NUM_VMS-1, indpb=1.0/NUM_TASKS)
toolbox.register("select", tools.selNSGA2)
toolbox.register("select_tournament", tools.selTournamentDCD)

POP_SIZE = 80
NGEN = 100

pop = toolbox.population(n=POP_SIZE)
invalid = [ind for ind in pop if not ind.fitness.valid]
fits = toolbox.map(toolbox.evaluate, invalid)
for ind, fit in zip(invalid, fits):
    ind.fitness.values = fit

for gen in range(NGEN):
    # Garantir que as distâncias de crowding estejam atribuídas para cada front
    # antes de usar selTournamentDCD, que depende de fitness.crowding_dist.
    from deap.tools.emo import assignCrowdingDist
    fronts = tools.sortNondominated(pop, len(pop))
    for front in fronts:
        assignCrowdingDist(front)

    offspring = toolbox.select_tournament(pop, len(pop))
    offspring = [toolbox.clone(ind) for ind in offspring]

    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.9:
            toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            del ind2.fitness.values

    for ind in offspring:
        if random.random() < 0.2:
            toolbox.mutate(ind)
            del ind.fitness.values

    invalid = [ind for ind in offspring if not ind.fitness.valid]
    fits = toolbox.map(toolbox.evaluate, invalid)
    for ind, fit in zip(invalid, fits):
        ind.fitness.values = fit

    pop = toolbox.select(pop + offspring, POP_SIZE)

pareto = tools.sortNondominated(pop, len(pop), first_front_only=True)[0]
pareto_vals = [ind.fitness.values for ind in pareto]
pareto_vals


AttributeError: 'FitnessMin' object has no attribute 'crowding_dist'

In [ ]:
# Plot interativo da fronteira de Pareto com Plotly
# Esta célula salva `pareto_plot.html` e `pareto_vals.csv`.
import numpy as np
import pandas as pd
import plotly.express as px

try:
    arr = np.array(pareto_vals)
except NameError:
    print("Variável pareto_vals não encontrada no escopo. Execute o notebook até o cálculo do pareto antes de rodar esta célula.")
else:
    if arr.size == 0:
        print('pareto_vals está vazio — nada para plotar')
    else:
        # garantir forma Nx2
        arr = arr.reshape(-1, 2)
        df = pd.DataFrame(arr, columns=['latency', 'cost'])

        fig = px.scatter(df, x='latency', y='cost',
                         title='Pareto front: latency vs cost',
                         labels={'latency': 'Mean latency', 'cost': 'Total cost'},
                         height=600, width=800)

        fig.update_traces(marker=dict(size=10, line=dict(width=1, color='DarkSlateGrey')))

        # salvar interativo como HTML
        fig.write_html('pareto_plot.html')
        print('Salvo: pareto_plot.html')

        # também salvar CSV
        df.to_csv('pareto_vals.csv', index=False)
        print('Salvo: pareto_vals.csv')

        # mostrar no ambiente que estiver abrindo o notebook
        fig.show()
